In [0]:
# Quitar comentarios para ejecutar online
# import sys
# !{sys.executable} -m pip install seaborn
# !{sys.executable} -m pip install statsmodels

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import normalize

from scipy import stats

from statsmodels.stats.outliers_influence import variance_inflation_factor


# Desafío final 

## Preparado de datos

Habiendo estudiado el dataset recibido, hemos establecido el objetivo de buscar la mejor zona para establecer un nuevo restaurante y que tipo de comida se debería servir. Con el propósito de triunfar entre los  usuarios que frecuentan la página web de donde provienen los datos.





### Division de los negocios en  restaurantes

Con el objetivo establecido empezaremos el tratamiento de datos. Lo primero que haremos es seleccionar los negocios que sean restaurantes. Para hacer esta distinción hemos utilizado el atributo categoría, ya que contienen los tags que definen a cada uno de los negocios. Hemos escogido los negocios que tengan en sus categorías el valor “Restaurants”.


In [0]:
negocios = pd.read_csv("./Files/datos_negocios.csv")
negocios = negocios.dropna(how="any")
negocios[negocios["categorias"].str.contains("Restaurants")].to_csv("./Files/restaurants_only.csv")

### Renombrado de categorias

Ya dividido los datos, nos encontramos un problema a la hora de analizar el atributo categorías.  Este atributo es un string que contienen los diferentes tags que definen a los negocios. Esto nos impide analizar correctamente este atributo ya que existen más de 10000 valores únicos. Además se añade la condición de que solo queremos saber el tipo comida que se sirve en el restaurante. Para solventar este problema hemos analizado los diferentes tags existentes en el dataset y hemos hecho una selección de 100 de estos. Los 100 tags contienen  los diferentes tipos de comida que se sirven en cada uno de los restaurantes. Con estos tags ya seleccionado hemos sustituido los valores del atributo categorías por el correspondiente.

In [0]:
restaurantes = pd.read_csv("./Files/restaurants_only.csv", sep=",")
categorias = pd.read_csv("./Files/categorias.csv", sep=",").categorias.tolist()
tag = []
for i in restaurantes.categorias:
    size = len(tag)
    for k in i.split(","):
        if k[0] == " ":
            k = k[1:]
        if k in categorias:
            tag.append(k)
            break
    if size == len(tag):
        tag.append("nan")

restaurantes.categorias = tag
restaurantes[restaurantes["categorias"] != "nan"].to_csv("./Files/restaurants_only.csv")

### Selección de opiniones de los restaurantes.
Del fichero de opiniones solo obtendremos las que se hayan realizado a los restaurantes escogidos anteriormente. Este fichero al ser de gran tamaño hemos decidido solo escoger los primeros tres millones de instancias hacer un shuffle y escoger las primeras cien mil instancias que contengan los id de los restaurantes. La decisión de escoger cien mil instancias se ha tomado para reducir el tiempo de ejecución. Se ha eliminado el atributo texto de la opinión debido a su irrelevancia en el análisis y al gran aporte de bytes que ofrece al fichero.




In [0]:
opinions = pd.read_csv("./Files/datos_opiniones.csv", delimiter=",", nrows=4000000)
restaurante = pd.read_csv("./Files/restaurants_only.csv")
opinions = opinions.drop("texto", axis=1)
opinions[opinions["negocio_id"].isin(restaurante["negocio_id"])].sample(frac=1).sort_values(ascending = False,by="fecha").head(1500000).to_csv("./Files/opiniones_restaurantes.csv")

### Selección de usuarios

Respecto a los usuarios escogeremos aquellos que participen el subconjunto de opiniones escogidas anteriormente. Se ha escogido este subconjunto ya que  solo nos interesan los usuarios que opinan en los restaurantes seleccionados.

In [0]:
opiniones = pd.read_csv("Files/opiniones_restaurantes.csv", sep=",")
users = pd.read_csv("Files/datos_usuarios_noAmigos.csv", sep=",")
users[users["usuario_id"].isin(opiniones["usuario_id"].unique())].to_csv("./Files/datos_usuarios_restaurante.csv")